# content

## Model manipulations

this is just a collection of working code snippets to show basic operations when using OR-Tools

c.f. [github OR-tools](https://github.com/google/or-tools/blob/stable/ortools/sat/docs/model.md) for more details

## used example

in order to have running code we use the chinese remainder theorem  as a simple example, i.e. determine the remainder of the division of a given number of (coprime) divisors.

In [ ]:
from ortools.sat.python import cp_model

In [ ]:
data = {
    'army_ub': 10000,
    'counts': [13,19,37]
}

In [ ]:
# instantiate the model and the solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()
# declare variable (together with bounds)
army = model.NewIntVar(1, data['army_ub'], 'army')
# declare constraints
for i in data['counts']:
    model.AddModuloEquality(0, army, i)
# apply solver
status = solver.Solve(model)  
if status == cp_model.OPTIMAL:
    print("solution time %s milliseconds" %round(solver.WallTime(), 3))
    print("soldiers in enemy army: %s" %solver.Value(army))

solution time 0.007 milliseconds
soldiers in enemy army: 9139


### changing bounds 

In the example we assumed that the upper bound for variable `army` eguals 10000. The following snippet shows, how to change a variable bound

In [ ]:
new_value = 100000
army.Proto().domain[:] = []
army.Proto().domain.extend(cp_model.Domain(1, new_value).FlattenedIntervals())
army.Proto()

name: "army"
domain: 1
domain: 100000

In [ ]:
class CallbackSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print all solutions."""

    def __init__(self, variable):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variable = variable

    def on_solution_callback(self):
        print(self.Value(self.__variable))

In [ ]:
solutionprinter = CallbackSolutionPrinter(army)
solver.parameters.enumerate_all_solutions = True
solver_status = solver.Solve(model, solutionprinter)

9139
18278
27417
36556
45695
54834
63973
73112
82251
91390
